In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./footnotes_cik_train_rp.csv')
data.head()

,Unnamed: 0,filing_accessionNumber,name,category,textblock_id,word_count_only_text,readable_text,cikNumber,cikn,clean_text
0,0,0000097134-17-000006,AccountsPayableAccruedLiabilitiesAndOtherLiabi...,Liabilities,452,24,"Other Current Liabilities as of December 31, c...",97134,97134.0,other current liabilities as of december consi...
1,1,0001378950-15-000019,CommitmentsAndContingenciesDisclosureTextBlock,Commitments_Contingencies,11136,821,Commitments and Contingencies Facilities Lease...,1378950,1378950.0,commitments and contingencies facilities lease...
2,2,0001437749-14-004605,FinancialInstrumentsDisclosureTextBlock,Derivatives_Hedging,9450,715,NOTE 17 - FAIR VALUE OF FINANCIAL INSTRUMENTS...,1087456,1087456.0,fair value of financial instruments the carryi...
3,4,0001564590-17-001729,BusinessCombinationDisclosureTextBlock,Business_Combination,3057,466,3. Business Acquisitions For the year ended D...,1492633,1492633.0,business acquisitions for the year ended decem...
4,5,0000913165-17-000006,BusinessCombinationDisclosureTextBlock,Business_Combination,3446,736,BUSINESS ACQUISITIONS Exactech Australia On Fe...,913165,913165.0,business acquisitions exactech australia on fe...


In [7]:
train_data = data.loc[:,['category', 'clean_text']][:1000]

In [8]:
categories = list(train_data.category.unique())
instruction = f"""Among following 32 categories: {categories}
        Predict the category of the input sentence"""



In [9]:
dataset_data = [
    {
        "instruction": instruction,
        "input": row_dict["clean_text"],
        "output": row_dict["category"]
    }
    for row_dict in train_data.to_dict(orient="records")
]

In [11]:
import json
with open("alpaca-category-dataset.json", "w") as f:
   json.dump(dataset_data, f)

In [12]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer
import os
import sys
from typing import List
 
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
 
import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
from pylab import rcParams
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

/home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [13]:
BASE_MODEL = "../Llama-2-7b-chat-hf"
 
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
 

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.92s/it]


In [14]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
 
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

In [15]:
data = load_dataset("json", data_files="alpaca-category-dataset.json")
data["train"]

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 211.68it/s]


Dataset json downloaded and prepared to /home/rawat.m/.cache/huggingface/datasets/json/default-a3db3d08b56743a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 44.52it/s]


Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 1000
})

In [16]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
 
 
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [17]:
train_val = data["train"].train_test_split(
    test_size=200, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]
 
BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

In [19]:
model = prepare_model_for_int8_training(model)

/home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [20]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

In [21]:
model = get_peft_model(model, config)

In [22]:
model.print_trainable_parameters()



trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [23]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [24]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [25]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)

In [26]:
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

In [27]:
model = torch.compile(model)


In [28]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/transformers/trainer.py:1645 in train │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/transformers/trainer.py:1938 in       │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/transformers/trainer.py:2770 in       │
│ training_step                                                                                    │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /home/rawat.m/.conda/envs/llm/lib/python3.10/site-packages/accelerate/accelerator.py:1819 in     │
│ backward                                                   